In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import datetime as dt
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [4]:
department = os.listdir('./Выгрузки')

In [5]:
#Отчет для ежедневного контроля менеджеров
for dep in tqdm(department):   
    #из СПДП вытаскиваем уникальных сотрудников
    spdp = pd.read_excel(f'./Выгрузки/{dep}/СПДП.xlsx', header=3)
    spdp.drop_duplicates(keep='first', inplace=True) 
    spdp.dropna(axis='columns', how='all', inplace=True)
    spdp = spdp[spdp['Сотрудник ДП.Контрагенты доступ.Не учитывать в ЗП'] == 'Нет']

    mp_code=spdp[['Сотрудник ДП.ФИО Сотрудника', 'Сотрудник ДП.Контрагенты доступ.Контрагент.Код', 'Сотрудник ДП.Контрагенты доступ.Не учитывать в ЗП', 'Непосредственный руководитель']]
    df = pd.DataFrame(mp_code)
    df.rename(columns={'Сотрудник ДП.ФИО Сотрудника':'МП',
                        'Сотрудник ДП.Контрагенты доступ.Контрагент.Код':'Код контрагента'}, inplace=True)

    #население
    people = pd.read_excel(f'./Выгрузки/{dep}/Сборка населения_23.xlsx', header=0)    
    df = df.merge(people[['Код группы', 'Код папки', 'Наименование', 'Население']], how='left', left_on='Код контрагента', right_on='Код группы') 
    df = df.merge(people[['Код папки', 'Наименование', 'Население']], how='left', left_on='Код контрагента', right_on='Код папки')  

    #если код папки_х = 0, то заполнить значением из столбца код папки_у
    df.loc[df['Код папки_x'].isna() == True, 'Код папки_x'] = df['Код папки_y']
    df.loc[df['Наименование_x'].isna() == True, 'Наименование_x'] = df['Наименование_y']
    df.loc[df['Население_x'].isna() == True, 'Население_x'] = df['Население_y']

    #продажи 22 года и 23 года
    sales_22 = pd.read_excel(f'./Выгрузки/{dep}/22.xlsx', header=7)
    df = df.merge(sales_22, how='left', left_on='Код контрагента', right_on='Код')
    df.rename(columns={'Сумма продажи в руб.':'Оборот 22', 'Количество клиентов':'Клиенты 22', 'Контрагент':'Контрагент территория'}, inplace=True)
    df.drop_duplicates(subset=['Код контрагента'], keep='first', inplace=True)  

    sales_23 = pd.read_excel(f'./Выгрузки/{dep}/23.xlsx', header=7)
    df = df.merge(sales_23, how='left', left_on='Код контрагента', right_on='Код')
    df.rename(columns={'Сумма продажи в руб.':'Оборот 23', 'Количество клиентов':'Клиенты 23'}, inplace=True)
    
    #продажи СТМ 22 года и 23 года
    sales_CTM_22 = pd.read_excel(f'./Выгрузки/{dep}/стм22.xlsx', header=7)
    df = df.merge(sales_CTM_22, how='left', left_on='Код контрагента', right_on='Код')
    df.rename(columns={'Сумма продажи в руб.':'Оборот СТМ 22', 'Количество клиентов':'Клиенты СТМ 22', 'Код':'Код2', 'Unnamed: 0':'Unnamed'}, inplace=True)

    sales_CTM_23 = pd.read_excel(f'./Выгрузки/{dep}/стм23.xlsx', header=7)
    df = df.merge(sales_CTM_23, how='left', left_on='Код_x', right_on='Код')
    df.rename(columns={'Сумма продажи в руб.':'Оборот СТМ 23', 'Количество клиентов':'Клиенты СТМ 23', 'Непосредственный руководитель':'Руководитель'}, inplace=True)   

    #регион и руководитель
    slovar = pd.read_excel(f'./Выгрузки/{dep}/Словарь руководителей и регионов.xlsx', header=0)
    df = df.merge(slovar, how='left', left_on='Руководитель', right_on='Непосредственный руководитель/сервис-менеджер')
    
    #план/факт продаж
    pf = pd.read_excel(f'./Выгрузки/{dep}/ПФ.xlsx', header=17)
    df = df.merge(pf[['Руководитель',
                      'План',
                      'Факт',
                      'Процент выполнения',
                      'Сумма на сборке',
                      'Сумма факт и сборка',
                      'Процент по сумма факт и сборка',
                      'Сумма резерв',
                      'Сумма факт и сборка и резерв',
                      'Процент по сумма факт и сборка и резерв']], how='left', left_on='МП', right_on='Руководитель')
    
    #замена пустых значение в колонке "План" на 0. Сохраним в df только те строки, где значения в ПЛАН >0
    df['План'].fillna(0)
    df = df[df['План'] > 0]    
    
    #спеццель
    spec = pd.read_excel(f'./Выгрузки/{dep}/спеццель.xlsx', header=5)
    spec.rename(columns={'План':'План_спец', 'Факт':'Факт_спец', '% факт':'% выполнения_спец'}, inplace=True)
    df = df.merge(spec[['Сотрудник', 'План_спец', 'Факт_спец', '% выполнения_спец']], how='left', left_on='МП', right_on='Сотрудник')  

    #звонки
    call = pd.read_excel(f'./Выгрузки/{dep}/Звонки.xlsx', header=5)
    df = df.merge(call[['Сотрудник ДП', 'Количество записей потенциал', 'Всего звонков']], how='left', left_on='МП', right_on='Сотрудник ДП')  

    #удаляем дубли в плане, факте и тд
    plan = df[['МП', 'План','Факт','Сумма на сборке','Сумма факт и сборка','Сумма резерв','Сумма факт и сборка и резерв','План_спец','Факт_спец', 'Количество записей потенциал', 'Всего звонков']]
    plan.drop_duplicates(subset='МП', inplace=True)
    fg = df.merge(plan, how='left', left_index=True, right_index=True)

    #новые клиенты и возвращенные клиенты
    nk = pd.read_excel(f'./Выгрузки/{dep}/НК.xlsx', header=4)
    fg = fg.merge(nk[['Менеджер', 'Факт', 'Сумма реализаций']], how='left', left_on='МП_y', right_on='Менеджер')
    fg.rename(columns={'Сумма реализаций':'НК сумма'}, inplace=True)

    vk = pd.read_excel(f'./Выгрузки/{dep}/ВК.xlsx', header=4)
    fg = fg.merge(vk[['Возвращено', 'Сумма реализаций', 'Сотрудник']], how='left', left_on='МП_y', right_on='Сотрудник')
    fg.rename(columns={'Сумма реализаций':'ВК сумма'}, inplace=True)

    fg = fg.merge(people[['Код группы', 'Наименование', 'Население']], how='left', left_on='Код папки_x', right_on='Код группы')
    fg.rename(columns={'Код группы_y':'Код группы_2', 'Наименование':'Наименование_2', 'Население':'Население_2'}, inplace=True)

    fg2 = fg[['МП_x',
              'МП_y',
              'Регион',
              'Непосредственно руководитель (линейник)',
              'Код контрагента',
              'Контрагент территория',
              'Клиенты 22',
              'Оборот 22',
              'Клиенты 23',
              'Оборот 23',
              'Оборот СТМ 22',
              'Клиенты СТМ 22',
              'Оборот СТМ 23',
              'Клиенты СТМ 23',
              'План_x',
              'Факт_x',
              'Код папки_x',
              'Наименование_x',
              'Население_x',
              'Сумма на сборке_x',
              'Сумма факт и сборка_x',
              'Сумма резерв_x',
              'Сумма факт и сборка и резерв_x',
              'План_спец_x',
              'Факт_спец_x',
              'Количество записей потенциал_x',
              'Всего звонков_x',
              'НК сумма',
              'ВК сумма',
              'Код группы_2',
              'Наименование_2',
              'Население_2']]
    
    #удалить дубли 
    fg2['МП_y'][fg2.duplicated(['МП_y']) == True] = np.NaN
    fg2['Клиенты 22'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Клиенты 23'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Оборот 23'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Оборот СТМ 22'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Клиенты СТМ 22'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Оборот СТМ 23'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Клиенты СТМ 23'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Население_x'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
    fg2['Оборот 22'][fg2.duplicated(['Оборот 22']) == True] = np.NaN
 
    fg2['Сумма на сборке_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Сумма факт и сборка_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Сумма резерв_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Сумма факт и сборка и резерв_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Сумма резерв_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['План_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Факт_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['План_спец_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Факт_спец_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Количество записей потенциал_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Всего звонков_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Всего звонков_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    fg2['Всего звонков_x'][fg2.duplicated(['МП_x']) == True] = np.NaN
    
    fg2.to_excel(f'./Готовый ДЭШ по ДП/{dep}.xlsx', index=False)
#объединение всех файлов в 1
all_s=Path(f'./Готовый ДЭШ по ДП')
fg2=(pd.concat([pd.read_excel(f).assign(file_name=f.stem) for f in all_s.glob('*xlsx')], ignore_index=True))

fg2.to_excel(f'./dashboard.xlsx', index=False)

  0%|          | 0/1 [00:00<?, ?it/s]100%|██████████| 1/1 [00:08<00:00,  8.06s/it]
